---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [42]:
import pandas as pd
import numpy as np
import nltk

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0) #just like "20 > 3 ? true : false" in JAVA
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [0]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [0]:
def answer_one():
    
    return (len(spam_data[spam_data['target']==1])/len(spam_data))*100

In [8]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    vocab = vect.get_feature_names()
    m = 0
    for f in vocab:
        if m < len(f):
            m = len(f)
            k = f
            
    return k

In [10]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    
    model = MultinomialNB(0.1)
    model.fit(vect.transform(X_train), y_train)   
    
    return roc_auc_score(y_test, model.predict(vect.transform(X_test)))

In [184]:
answer_three()

0.9581366823421557

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    import operator

    vect = TfidfVectorizer()
    X_train_transformed = vect.fit_transform(X_train)

    feature_names = vect.get_feature_names()
    idfs = vect.idf_
    names_idfs = list(zip(feature_names, idfs))

    smallest = sorted(names_idfs, key=operator.itemgetter(1))[:20]
    smallest = pd.Series([features[1] for features in smallest], index=[features[0] for features in smallest])

    largest = sorted(names_idfs, key=operator.itemgetter(1), reverse=True)[:20]
    largest = pd.Series([features[1] for features in largest], index=[features[0] for features in largest])
    
    return (smallest, largest)

In [14]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64, 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
def answer_five():
    
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict(vect.transform(X_test))    
    
    return roc_auc_score(y_test, predictions)

In [18]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [0]:
def answer_six():
    
   spam_data['length'] = spam_data['text'].str.len() 
    
   return (np.mean(spam_data['length'][spam_data['target'] == 0]), np.mean(spam_data['length'][spam_data['target'] == 1]))

In [111]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [0]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.svm import SVC

def answer_seven():
  
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    
    X_train_vectorized = vect.transform(X_train)

    model = SVC(kernel='linear', C=10000)
    model.fit(add_feature(X_train_vectorized, X_train.str.len()), y_train)

    predictions = model.predict(add_feature(vect.transform(X_test), X_test.str.len()))

    return roc_auc_score(y_test, predictions)
    

In [113]:
answer_seven()

0.9522838395327912

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [0]:
def answer_eight():
    
    spam_data['digit_length'] = spam_data['text'].str.findall(r'\d').str.len()
    
    return (np.mean(spam_data['digit_length'][spam_data['target']==0]),
           np.mean(spam_data['digit_length'][spam_data['target']==1]))

In [141]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
  
  vect = TfidfVectorizer(ngram_range=(1,3), min_df=5).fit(X_train)
  
  X_train_vectorized_with_extra_features = add_feature(vect.transform(X_train),
                                                       [X_train.str.len(), X_train.str.findall(r'\d').str.len()])
  
  X_test_vectorized_with_extra_features = add_feature(vect.transform(X_test),
                                                     [X_test.str.len(), X_test.str.findall(r'\d').str.len()])
  
  
  model = LogisticRegression(C=100).fit(X_train_vectorized_with_extra_features, y_train)
  
  predictions = model.predict(X_test_vectorized_with_extra_features)
    
    
  return roc_auc_score(y_test, predictions)

In [135]:
answer_nine()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [0]:
def answer_ten():
    
    spam_data['non_words'] = spam_data['text'].str.findall('[\W]').str.len()
    
    return (np.mean(spam_data['non_words'][spam_data['target']==0]),
           np.mean(spam_data['non_words'][spam_data['target']==1]))

In [153]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [0]:
def answer_eleven():
  
  vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train)
  
  # regex: \d - for all digits, \W - for all non-words
  X_train_vect_with_extra_features = add_feature(vect.transform(X_train),
                                                [X_train.str.len(),                                                 
                                                 X_train.str.findall('\d').str.len(),
                                                 X_train.str.findall('\W').str.len()])
  
  X_test_vect_with_extra_features = add_feature(vect.transform(X_test),
                                                [X_test.str.len(),                                                 
                                                 X_test.str.findall('\d').str.len(),
                                                 X_test.str.findall('\W').str.len()])
  
  model = LogisticRegression(C=100).fit(X_train_vect_with_extra_features, y_train)
  
  predictions = model.predict(X_test_vect_with_extra_features)
  
  auc_score = roc_auc_score(y_test, predictions)
  
  #coefficient extraction
  feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
  sorted_coeffs_indices = model.coef_[0].argsort() #argsort: returns ascending ordered indices of given list
  
  smallest = feature_names[sorted_coeffs_indices[:10]]
  largest = feature_names[sorted_coeffs_indices[:-11:-1]] #-1 step argument,which reverses the order of array, -11 returns last 10 items from the array
    
    
  return (auc_score, list(smallest), list(largest))

In [216]:
answer_eleven()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])